<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Concurrency-with-Shared-Variables" data-toc-modified-id="Concurrency-with-Shared-Variables-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Concurrency with Shared Variables</a></span><ul class="toc-item"><li><span><a href="#Race-Conditions" data-toc-modified-id="Race-Conditions-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>Race Conditions</a></span></li><li><span><a href="#Mutual-Exclusion:-sync.Mutex" data-toc-modified-id="Mutual-Exclusion:-sync.Mutex-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>Mutual Exclusion: sync.Mutex</a></span></li><li><span><a href="#Read/Write-Mutexes:-sync.RWMutex" data-toc-modified-id="Read/Write-Mutexes:-sync.RWMutex-9.3"><span class="toc-item-num">9.3&nbsp;&nbsp;</span>Read/Write Mutexes: sync.RWMutex</a></span></li><li><span><a href="#Memory-Synchronization" data-toc-modified-id="Memory-Synchronization-9.4"><span class="toc-item-num">9.4&nbsp;&nbsp;</span>Memory Synchronization</a></span></li><li><span><a href="#Lazy-Initialization:-sync.Once" data-toc-modified-id="Lazy-Initialization:-sync.Once-9.5"><span class="toc-item-num">9.5&nbsp;&nbsp;</span>Lazy Initialization: sync.Once</a></span></li><li><span><a href="#The-Race-Detector" data-toc-modified-id="The-Race-Detector-9.6"><span class="toc-item-num">9.6&nbsp;&nbsp;</span>The Race Detector</a></span></li><li><span><a href="#Example:-Concurrent-Non-Blocking-Cache" data-toc-modified-id="Example:-Concurrent-Non-Blocking-Cache-9.7"><span class="toc-item-num">9.7&nbsp;&nbsp;</span>Example: Concurrent Non-Blocking Cache</a></span></li><li><span><a href="#Goroutines-and-Threads" data-toc-modified-id="Goroutines-and-Threads-9.8"><span class="toc-item-num">9.8&nbsp;&nbsp;</span>Goroutines and Threads</a></span><ul class="toc-item"><li><span><a href="#Growable-Stacks" data-toc-modified-id="Growable-Stacks-9.8.1"><span class="toc-item-num">9.8.1&nbsp;&nbsp;</span>Growable Stacks</a></span></li><li><span><a href="#Groroutine-Scheduling" data-toc-modified-id="Groroutine-Scheduling-9.8.2"><span class="toc-item-num">9.8.2&nbsp;&nbsp;</span>Groroutine Scheduling</a></span></li><li><span><a href="#GOMAXPROCS" data-toc-modified-id="GOMAXPROCS-9.8.3"><span class="toc-item-num">9.8.3&nbsp;&nbsp;</span>GOMAXPROCS</a></span></li><li><span><a href="#Goroutines-Have-No-Identify" data-toc-modified-id="Goroutines-Have-No-Identify-9.8.4"><span class="toc-item-num">9.8.4&nbsp;&nbsp;</span>Goroutines Have No Identify</a></span></li></ul></li></ul></li></ul></div>

# Concurrency with Shared Variables

## Race Conditions

In [15]:
var balance int
func Deposit(amount int) {  balance = balance + amount }
func Balance() int {return  balance }

In [157]:
// Alice:
import ("time";"fmt")
%%
go func() {
    Deposit(200)                  // A1
    fmt.Println("=", Balance())   // A2
}()
// Bob:
go Deposit(100)                  //B
time.Sleep(100* time.Millisecond)//delay termination

= 300


In [91]:
// Bob:
%%
go Deposit(100)                  //B

In [160]:
%%
for i:=0; i<100; i++ {
    balance=0
    for j :=0; j<100; j++ {
      go Deposit(4)
       // Deposit(1)
    }
    time.Sleep(100* time.Millisecond)
    fmt.Println("=",Balance())
}

= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 392
= 392
= 364
= 400
= 400
= 400
= 368
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 352
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 388
= 364
= 400
= 400
= 400
= 400
= 400
= 400
= 388
= 400
= 400
= 392
= 400
= 400
= 400
= 372
= 400
= 400
= 384
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400
= 400


In [291]:
%%
var x []int
go func() { x = make([]int,10)}()
go func() { x = make([]int,1000000)}()
//time.Sleep(1* time.Millisecond)
x[999999] = 1 // Note: undefined behavior; memory corruption possible

panic: runtime error: index out of range [999999] with length 0

goroutine 1 [running]:
main.main()
	 [[ Cell [291] Line 6 ]] /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_cca68419/main.go:23 +0x13c
exit status 2


* A data race occurs whenever two goroutines access the same variable concurrently and at least one of the accesses is a write
* Three ways to avoid a data race
  * The first way: not to write the variable
  * The second way: avoid acessing the variable from multiple goroutines
    * Do not communicate by sharing memory; share memory by communicating
  * The third way: allow many goroutines to access the variable, but only one at a time
    * mutual exclusion

In [327]:
// The first way:not to write the variable
import "image"
var icons = make(map[string]image.Image)
func loadIcon(name string) image.Image
// not concurrency-safe
func Icon(name string) image.Image {
    icon, ok := icons[name]
    if !ok {
        icon = loadIcon(name)
        icons[name] = icon
    }
    return icon
}

# gonb_cca68419 

 

 Cell[327]: Line 4 
 ./main.go:81:6: missing function body
 
	go teller() // start the monitor goroutine
}

 func loadIcon(name string) image.Image
 
func teller() {

ERROR: failed to run "/opt/homebrew/bin/go build -o /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_cca68419/gonb_cca68419": exit status 1

In [326]:
import "image"
var icons = map[string]image.Image {
    "spades.png":  loadIcon("spades.png"),
    "hearts.png":  loadIcon("hearts.png"),
    "diamonds.png":loadIcon("diamonds.png"),
    "clubs.png":   loadIcon("clubs.png"),
}
// Concurrency-safe,
func Icon(name string) image.Image {return icons[name]}

# gonb_cca68419 

 

 Cell[326]: Line 3 
 ./main.go:18:20: undefined: loadIcon
 
	balances = make(chan int)
	deposits = make(chan int)
	icons = map[string]image.Image {
 "spades.png": loadIcon("spades.png"),
 "hearts.png": loadIcon("hearts.png"),
 "diamonds.png":loadIcon("diamonds.png"),

 

 

 Cell[326]: Line 4 
 ./main.go:19:20: undefined: loadIcon
 
	deposits = make(chan int)
	icons = map[string]image.Image {
 "spades.png": loadIcon("spades.png"),
 "hearts.png": loadIcon("hearts.png"),
 "diamonds.png":loadIcon("diamonds.png"),
 "clubs.png": loadIcon("clubs.png"),

 

 

 Cell[326]: Line 5 
 ./main.go:20:20: undefined: loadIcon
 
	icons = map[string]image.Image {
 "spades.png": loadIcon("spades.png"),
 "hearts.png": loadIcon("hearts.png"),
 "diamonds.png":loadIcon("diamonds.png"),
 "clubs.png": loadIcon("clubs.png"),
}

 

 

 Cell[326]: Line 6 
 ./main.go:21:20: undefined: loadIcon
 
 "spades.png": loadIcon("spades.png"),
 "hearts.png": loadIcon("hearts.png"),
 "diamonds.png":loadIcon("diamonds.png"),
 "clubs.png": loadIcon("clubs.png"),
 }
	sema = make (chan struct{},1)

ERROR: failed to run "/opt/homebrew/bin/go build -o /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_cca68419/gonb_cca68419": exit status 1

* The second way
  * Use a channel to send the confining goroutine a request to query or update the variable
    * monitor goroutine: broker access to a confined variable using channel requests 

In [325]:
//gopl.io/ch9/bank1
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 261.
//!+

// Package bank provides a concurrency-safe bank with one account.
//package bank

var deposits = make(chan int) // send amount to deposit
var balances = make(chan int) // receive balance

func Deposit(amount int) { deposits <- amount }
func Balance() int       { return <-balances }

func teller() {
	var balance int // balance is confined to teller goroutine
	for {
		select {
		case amount := <-deposits:
			balance += amount
		case balances <- balance:
		}
	}
}

func init() {
	go teller() // start the monitor goroutine
}

//!-

In [323]:
//back_test
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

//package bank_test

import (
	"fmt"
	"testing"

	"gopl.io/ch9/bank1"
)

func TestBank(t *testing.T) {
	done := make(chan struct{})

	// Alice
	go func() {
		bank.Deposit(200)
		fmt.Println("=", bank.Balance())
		done <- struct{}{}
	}()

	// Bob
	go func() {
		bank.Deposit(100)
		done <- struct{}{}
	}()

	// Wait for both transactions.
	<-done
	<-done

	if got, want := bank.Balance(), 300; got != want {
		t.Errorf("Balance = %d, want %d", got, want)
	}
}

In [324]:
%%
var t testing.T
TestBank(&t)

= 300


In [321]:
import "fmt"
type Cake struct {state string}
func baker(cooked chan <- *Cake) {
    for {
      cake := new(Cake)
      cake.state = "cooked"
      fmt.Println(cake.state)
      cooked <- cake // baker never touches this cake again
    }
}

func icer(iced chan <- *Cake, cooked <- chan *Cake){
    for cake := range cooked {
        cake.state = "iced"
        fmt.Println(cake.state)
        iced <- cake //icer never touches this cake aain
    }
}

In [322]:
// Cakes are serially confied, first to the baker, then to the icer
import "time"
%%
cooked := make(chan *Cake,1)
iced := make(chan *Cake,1)
go icer(iced, cooked)
go baker(cooked)
time.Sleep(5* time.Millisecond)

cooked
cooked
cooked
iced
iced
cooked


## Mutual Exclusion: sync.Mutex

In [328]:
//gopl.io/ch9/bank2
var (
    sema = make (chan struct{},1)
    balance int
)
func Deposit(amount int) {
    sema <- struct{}{} // acquire token
    balance = balance + amount
    <- sema  //release token
}
func Balance() int {
    sema <- struct{}{} // acquire token
    b := balance
    <- sema // release token
    return b
}

In [329]:
%%
for i:=0; i<10; i++ {
    balance=0
    for j :=0; j<100; j++ {
      go Deposit(3)
    }
    time.Sleep(100* time.Millisecond)
    fmt.Println("=",Balance())
}

= 300
= 300
= 300
= 300
= 300
= 300
= 300
= 300
= 300
= 300


In [330]:
//gopl.io/ch9/bank3
import "sync"
var (
  mu sync.Mutex //guar balance
  balance int
)

func Deposit(amount int) {
    mu.Lock()
    balance = balance + amount
    mu.Unlock()
}
func Balance() int {
    mu.Lock()
    b := balance
    mu.Unlock()
    return b
}

In [332]:
import ("time";"fmt")
%%
for i:=0; i<10; i++ {
    balance=0
    for j :=0; j<100; j++ {
      go Deposit(10)
    }
    time.Sleep(100* time.Millisecond)
    fmt.Println("=",Balance())
}

= 1000
= 1000
= 1000
= 1000
= 1000
= 1000
= 1000
= 1000
= 1000
= 1000


In [333]:
func Balance() int {
    mu.Lock()
    defer mu.Unlock()
    return balance
}

* Withdraw function, three implementation
  * without lock
  * lock a second time and lead to a deadlock
  * internal implementation without second lock

In [334]:
// Not Atomic
func Withdraw(amount int) bool {
    Deposit(-amount)
    if Balance() < 0 {
        Deposit(amount)
        return false // insufficient funds
    }
    return true
}

In [335]:
// incorrect, may be a deadlock
func Withdraw(amount int) bool {
    mu.Lock()
    defer mu.Unlock()
    Deposit(-amount) 
    if Balance() < 0 {
        Deposit(amount)
        return false // insufficient funds
    }
    return true
}

In [336]:
func Withdraw(amount int) bool {
    mu.Lock()
    defer mu.Unlock()
    deposit(-amount)
    if Balance() < 0 {
        deposit(amount)
        return false // insufficient funds
    }
    return true
}
func Deposit(amount int) {
    mu.Lock()
    defer mu.Unlock()
    deposit(amount)
}
func Balance() int {
    mu.Lock()
    defer mu.Unlock()
    return balance
}
// require the lock be held
func deposit(amount int) { balance += amount}

## Read/Write Mutexes: sync.RWMutex
* Multiple readers, single writer lock, provided with sync.RWMutex

In [337]:
import "sync"
var (
    mu sync.RWMutex
    balance int
)
func Deposit(amount int) {
    mu.Lock()
    balance = balance + amount
    mu.Unlock()
}

func Balance()int {
    mu.RLock()  // readers lock
    defer mu.RUnlock()
    return balance
}

In [338]:
import ("time";"fmt")
%%
for i:=0; i<10; i++ {
    balance=0
    for j :=0; j<100; j++ {
      go Deposit(1)
    }
    time.Sleep(100* time.Millisecond)
    fmt.Println("=",Balance())
}

= 100
= 100
= 100
= 100
= 100
= 100
= 100
= 100
= 100
= 100


## Memory Synchronization

In [339]:
%%
var x, y int

for i:=0; i< 10; i++ {
 x, y = 0,0
 go func(){
   x = 1                 // A1
   fmt.Print("y:",y," ") // A2
 }()

 go func(){
    y = 1                //B1
    fmt.Print("x:",x," ")//B2
 }()
 time.Sleep(300 * time.Millisecond)
 fmt.Println("")
 
}



y:0 x:1 
x:0 y:1 
x:0 y:1 
x:0 y:1 
y:0 x:1 
x:0 y:1 
x:0 y:1 
x:0 y:1 
x:0 y:1 
x:0 y:1 


## Lazy Initialization: sync.Once
* Defer an expensive initialization step until the moment it is needed

In [340]:
//Lazy initialization 
import "image"
var icons map[string]image.Image
func loadIcon(string)image.Image
func loadIcons(){
    icons = map[string]image.Image {
        "spades.png": loadIcon("spades.png"),
        "hearts.png": loadIcon("hearts.png"),
        "diamonds.png":loadIcon("diamonds.png"),
        "clubs.png":   loadIcon("clubs.png"),
    }
}
// Not concurrency-safe
func Icon(name string) image.Image {
    if icons == nil {
        loadIcons() // one-time initialization
    }
    return icons[name]
}

# gonb_cca68419 

 

 Cell[340]: Line 4 
 ./main.go:102:6: missing function body
 
	go teller() // start the monitor goroutine
}

 func loadIcon(string)image.Image
 
func loadIcons(){

ERROR: failed to run "/opt/homebrew/bin/go build -o /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_cca68419/gonb_cca68419": exit status 1

In [344]:
//icons may not be nil, but the initialization is not complete
import "image"
var icons map[string]image.Image
func loadIcons() {
    icons = make(map[string]image.Image)
    icons["spades.png"] = loadIcon("spades.png")
    icons["hearts.png"] = loadIcon("hearts.png")
    icons["diamonds.png"]=loadIcon("diamonds.png")
    icons["clubs.png"] = loadIcon("clubs.png")
}

# gonb_cca68419 

 

 Cell[344]: Line 6 
 ./main.go:97:27: undefined: loadIcon
 

func loadIcons() {
 icons = make(map[string]image.Image)
 icons["spades.png"] = loadIcon("spades.png")
 icons["hearts.png"] = loadIcon("hearts.png")
 icons["diamonds.png"]=loadIcon("diamonds.png")

 

 

 Cell[344]: Line 7 
 ./main.go:98:27: undefined: loadIcon
 
func loadIcons() {
 icons = make(map[string]image.Image)
 icons["spades.png"] = loadIcon("spades.png")
 icons["hearts.png"] = loadIcon("hearts.png")
 icons["diamonds.png"]=loadIcon("diamonds.png")
 icons["clubs.png"] = loadIcon("clubs.png")

 

 

 Cell[344]: Line 8 
 ./main.go:99:27: undefined: loadIcon
 
 icons = make(map[string]image.Image)
 icons["spades.png"] = loadIcon("spades.png")
 icons["hearts.png"] = loadIcon("hearts.png")
 icons["diamonds.png"]=loadIcon("diamonds.png")
 icons["clubs.png"] = loadIcon("clubs.png")
}

 

 

 Cell[344]: Line 9 
 ./main.go:100:26: undefined: loadIcon
 
 icons["spades.png"] = loadIcon("spades.png")
 icons["hearts.png"] = loadIcon("hearts.png")
 icons["diamonds.png"]=loadIcon("diamonds.png")
 icons["clubs.png"] = loadIcon("clubs.png")
 }

ERROR: failed to run "/opt/homebrew/bin/go build -o /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_cca68419/gonb_cca68419": exit status 1

In [342]:
// concurrency safe, but expensive for only one goroutine can access Icon
import "sync"
import "image"
var mu sync.Mutex // guard Icons
var icons map[string]image.Image
func Icon(name string)image.Image {
    mu.Lock()
    defer mu.Unlock()
    if icons == nil {
        loadIcons()
    }
    return icons[name]
}

# gonb_cca68419 

 

 Cell[337]: Line 13 
 ./main.go:25:8: mu.RLock undefined (type sync.Mutex has no field or method RLock)
 
)

func Balance()int {
 mu.RLock() // readers lock
 defer mu.RUnlock()
 return balance

 

 

 Cell[337]: Line 14 
 ./main.go:26:14: mu.RUnlock undefined (type sync.Mutex has no field or method RUnlock)
 

func Balance()int {
 mu.RLock() // readers lock
 defer mu.RUnlock()
 return balance
}

 

 

 Cell[342]: Line 10 
 ./main.go:40:9: undefined: loadIcons
 
 mu.Lock()
 defer mu.Unlock()
 if icons == nil {
 loadIcons()
 }
 return icons[name]

ERROR: failed to run "/opt/homebrew/bin/go build -o /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_cca68419/gonb_cca68419": exit status 1

In [351]:
var mu sync.RWMutex // guards icons
var icons map[string]image.Image
// Concurrency-safe
func Icon(name string) image.Image {
    mu.RLock()
    if icons != nil {
        icon := icons[name]
        mu.RUnlock()
        return icon
    }
    mu.RUnlock()
    //acquire an exclusive lock
    mu.Lock()
    if icons == nil {
        loadIcons()
    }
    icon := icons[name]
    mu.Unlock()
    return icon
}

# gonb_cca68419 

 

 Cell[351]: Line 15 
 ./main.go:61:9: undefined: loadIcons
 
 //acquire an exclusive lock
 mu.Lock()
 if icons == nil {
 loadIcons()
 }
 icon := icons[name]

ERROR: failed to run "/opt/homebrew/bin/go build -o /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_cca68419/gonb_cca68419": exit status 1

In [350]:
var loadIconsOnce sync.Once
var icons map[string]image.Image
//Concurrency-safe
func Icon(name string) image.Image {
    loadIconsOnce.Do(loadIcons)
    return icons[name]
}

# gonb_cca68419 

 

 Cell[350]: Line 5 
 ./main.go:52:22: undefined: loadIcons
 
}

func Icon(name string) image.Image {
 loadIconsOnce.Do(loadIcons)
 return icons[name]
}

ERROR: failed to run "/opt/homebrew/bin/go build -o /var/folders/4d/tg201k8x6yb81f_lrvt4440m0000gn/T/gonb_cca68419/gonb_cca68419": exit status 1

## The Race Detector

* Go runtime and toolchain are equipped with a dynamic analysis tool, the race detector
* Add the -race flat to go build, go run, or go test command
  * Record all accesses to shared variables that occurred during execution, along with the identify of the goroutinge that read or wrote the variable
  * Record all synchronization events
    * go statements, channel operations, and calls to (*sync.Mutex).Lock, (*sync.WaitGroup).wait, and so on.

## Example: Concurrent Non-Blocking Cache

In [345]:
// Package memo  1 provides a concurrency-unsafe
// memoization of a function of type Func.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 272.

//!+


//package memo

// A Memo caches the results of calling a Func.
type Memo struct {
	f     Func
	cache map[string]result
}

// Func is the type of the function to memoize.
type Func func(key string) (interface{}, error)

type result struct {
	value interface{}
	err   error
}

func New(f Func) *Memo {
	return &Memo{f: f, cache: make(map[string]result)}
}

// NOTE: not concurrency-safe!
func (memo *Memo) Get(key string) (interface{}, error) {
	res, ok := memo.cache[key]
	if !ok {
		res.value, res.err = memo.f(key)
		memo.cache[key] = res
	}
	return res.value, res.err
}

//!-

In [346]:
// Package memo  2 provides a concurrency-safe memoization a function of
// type Func.  Concurrent requests are serialized by a Mutex.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 275.


//package memo

import "sync"

// Func is the type of the function to memoize.
type Func func(string) (interface{}, error)

type result struct {
	value interface{}
	err   error
}

func New(f Func) *Memo {
	return &Memo{f: f, cache: make(map[string]result)}
}

//!+

type Memo struct {
	f     Func
	mu    sync.Mutex // guards cache
	cache map[string]result
}

// Get is concurrency-safe.
func (memo *Memo) Get(key string) (value interface{}, err error) {
	memo.mu.Lock()
	res, ok := memo.cache[key]
	if !ok {
		res.value, res.err = memo.f(key)
		memo.cache[key] = res
	}
	memo.mu.Unlock()
	return res.value, res.err
}

//!-

In [347]:
// Package memo 3 provides a concurrency-safe memoization a function of
// type Func.  Requests for different keys run concurrently.
// Concurrent requests for the same key result in duplicate work.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 276.


//package memo

import "sync"

type Memo struct {
	f     Func
	mu    sync.Mutex // guards cache
	cache map[string]result
}

type Func func(string) (interface{}, error)

type result struct {
	value interface{}
	err   error
}

func New(f Func) *Memo {
	return &Memo{f: f, cache: make(map[string]result)}
}

//!+

func (memo *Memo) Get(key string) (value interface{}, err error) {
	memo.mu.Lock()
	res, ok := memo.cache[key]
	memo.mu.Unlock()
	if !ok {
		res.value, res.err = memo.f(key)

		// Between the two critical sections, several goroutines
		// may race to compute f(key) and update the map.
		memo.mu.Lock()
		memo.cache[key] = res
		memo.mu.Unlock()
	}
	return res.value, res.err
}

//!-

In [348]:
// Package memo 4 provides a concurrency-safe memoization a function of
// a function.  Requests for different keys proceed in parallel.
// Concurrent requests for the same key block until the first completes.
// This implementation uses a Mutex.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 276.

//package memo
type Func func(string) (interface{}, error)

type result struct {
	value interface{}
	err   error
}

//
type entry struct {
	res   result
	ready chan struct{} // closed when res is ready
}

func New(f Func) *Memo {
	return &Memo{f: f, cache: make(map[string]*entry)}
}

type Memo struct {
	f     Func
	mu    sync.Mutex // guards cache
	cache map[string]*entry
}

func (memo *Memo) Get(key string) (value interface{}, err error) {
	memo.mu.Lock()
	e := memo.cache[key]
	if e == nil {
		// This is the first request for this key.
		// This goroutine becomes responsible for computing
		// the value and broadcasting the ready condition.
		e = &entry{ready: make(chan struct{})}
		memo.cache[key] = e
		memo.mu.Unlock()

		e.res.value, e.res.err = memo.f(key)

		close(e.ready) // broadcast ready condition
	} else {
		// This is a repeat request for this key.
		memo.mu.Unlock()

		<-e.ready // wait for ready condition
	}
	return e.res.value, e.res.err
}

//!-

In [349]:
// Package memo 5 provides a concurrency-safe non-blocking memoization
// of a function.  Requests for different keys proceed in parallel.
// Concurrent requests for the same key block until the first completes.
// This implementation uses a monitor goroutine.
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 278.

//package memo

//!+Func

// Func is the type of the function to memoize.
type Func func(key string) (interface{}, error)

// A result is the result of calling a Func.
type result struct {
	value interface{}
	err   error
}

type entry struct {
	res   result
	ready chan struct{} // closed when res is ready
}

//!-Func

//!+get

// A request is a message requesting that the Func be applied to key.
type request struct {
	key      string
	response chan<- result // the client wants a single result
}

type Memo struct{ requests chan request }

// New returns a memoization of f.  Clients must subsequently call Close.
func New(f Func) *Memo {
	memo := &Memo{requests: make(chan request)}
	go memo.server(f)
	return memo
}

func (memo *Memo) Get(key string) (interface{}, error) {
	response := make(chan result)
	memo.requests <- request{key, response}
	res := <-response
	return res.value, res.err
}

func (memo *Memo) Close() { close(memo.requests) }

//!-get

//!+monitor

func (memo *Memo) server(f Func) {
	cache := make(map[string]*entry)
	for req := range memo.requests {
		e := cache[req.key]
		if e == nil {
			// This is the first request for this key.
			e = &entry{ready: make(chan struct{})}
			cache[req.key] = e
			go e.call(f, req.key) // call f(key)
		}
		go e.deliver(req.response)
	}
}

func (e *entry) call(f Func, key string) {
	// Evaluate the function.
	e.res.value, e.res.err = f(key)
	// Broadcast the ready condition.
	close(e.ready)
}

func (e *entry) deliver(response chan<- result) {
	// Wait for the ready condition.
	<-e.ready
	// Send the result to the client.
	response <- e.res
}

//!-monitor

## Goroutines and Threads

### Growable Stacks
* a goroutine start with a small stack, typically 2KB
* a goroutine's stack is not fixed
  * it grows and shrinks as needed, with the limit of 1GB

### Groroutine Scheduling
* The Go runtime contains its own scheduler using a technique known as m:n scheduling
    * it multiplexes (or schedules) m go routines on n OS threads
    * the Go scheduler is not invoked periodically by a hardware timer
      * implicitly by certain Go language constructs
        * time.Sleep or blocks in a channel or mutex operation

### GOMAXPROCS
* GOMAXPROCS is the n in m:n scheduling

In [123]:
import "fmt"
for i:=1;i<100;i++ {
    go fmt.Print(0)
    fmt.Print(1)
}
// GOMAXPROC=1 go run hacker-cliche.go
// GOMAXPROC=2 go run hacker-cliche.go

111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

### Goroutines Have No Identify
* Goroutines have no notion of identity that is accessible to the programmer
* Go encourages a simpler style of programming in which parameters that affect the behavior of a function are explicit 
